In [ ]:
import pandas as pd
import numpy as np
import pymysql
import pymysql.cursors
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
%matplotlib inline

In [ ]:
#CHOOSING IMPORTANT FEATURES
from sklearn.feature_selection import SelectFromModel
import scipy.stats as stats
from scipy.stats import chi2_contingency
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error
from scipy.stats import skew
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_val_score
from itertools import product
from sklearn.metrics import r2_score
plt.rcParams['figure.figsize'] = (16.0, 10.0)

In [ ]:
import warnings
warnings.filterwarnings('ignore') # suppress warnings

In [ ]:
pd.describe_option('display')
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 500)

In [ ]:
full = pd.read_csv('full.csv',engine='python')

In [ ]:
full = full.drop(full.columns[full.columns.str.contains('unnamed',case = False)],axis = 1)

In [ ]:
full['WEIGHT'] = full['WEIGHT'].replace(1.000000e+16,np.nan)
full['WEIGHT'] = full['WEIGHT'].replace(' ',np.nan)
full['WEIGHT'] = full['WEIGHT'].fillna(9)
full['WEIGHT'] = full['WEIGHT'].astype(int)

In [ ]:
full.shape

In [ ]:
### deleting complete record afor a missing value
full = full.dropna()
full.shape

In [ ]:
full.isnull().values.sum()

In [ ]:
full.columns #no last_month

In [ ]:
full_c = pd.DataFrame(data = full, columns =['ACC_TYPE','BUS_USE','CDL_STAT','DAY','DRIMPAIR','DR_DRINK','DR_HGT','DR_SF1','DR_SF2','DR_SF3','DR_SF4',
          'EMER_USE','HARM_EV','HOUR','HAZ_PLAC','IMPACT1','LAST_YR','L_COMPL','L_RESTRI','L_STATUS','L_TYPE','MAKE','MAN_COLL',
          'MDRDSTRD','MDRMANAV','MFACTOR','MINUTE','MOD_YEAR','MONTH','MVIOLATN','MVISOBSC','OWNER','P_CRASH2','ROLLOVER','SPEC_USE',
          'SPEEDREL','STATE','TOW_VEH','VALIGN','VEH_SC1','VEH_SC2','VINTYPE','VPAVETYP','VPROFILE','VSURCOND','VTCONT_F','VTRAFCON'])

In [ ]:
full_c = full_c.apply(lambda x: x.astype('category')) #categories
#print(full_c.dtypes)

In [ ]:
full_o = pd.DataFrame(data = full, columns =['DR_WGT','GVWR','WEIGHT'])#.apply(lambda x: x.astype('category'), ordered = True))


In [ ]:
full_o = full_o.apply(lambda x: x.astype('category',ordered=True)) #Ordinal
#print(full_o.dtypes)

In [ ]:
full_p = pd.DataFrame(data = full, columns =['DEATHS','NUMOCCS','PREV_ACC','PREV_DWI','PREV_SPD','PREV_SUS','VE_FORMS','VNUM_LAN',
                                             'VTRAFWAY','FIRE_EXP','HAZ_INV','HIT_RUN','TRAV_SP'])
# count variables were left as is.

In [ ]:
full = pd.concat([full_o, full_c, full_p], axis = 1)

In [ ]:
vin_type = full.copy()
vin_type.head()

clean_up = {"VINTYPE" : {'P' : 0, "T" : 1, "M" : 2,"U" : 3,"C" : 4}}

vin_type.replace(clean_up, inplace = True)
print(vin_type.head())

full['VINTYPE'] = vin_type['VINTYPE']
full['VINTYPE'].head()

In [ ]:
full.shape #(296847, 63)

In [ ]:
cols = full.columns.tolist()
cols
cols = ['DR_WGT','GVWR','WEIGHT', 'ACC_TYPE', 'BUS_USE', 'CDL_STAT', 'DAY', 'DRIMPAIR', 'DR_DRINK', 'DR_HGT', 'DR_SF1', 'DR_SF2', 'DR_SF3',
        'DR_SF4','EMER_USE','HARM_EV','HOUR','HAZ_PLAC','IMPACT1','LAST_YR', 'L_COMPL', 'L_RESTRI', 'L_STATUS', 'L_TYPE', 'MAKE',
        'MAN_COLL','MDRDSTRD', 'MDRMANAV', 'MFACTOR', 'MINUTE', 'MOD_YEAR', 'MONTH', 'MVIOLATN', 'MVISOBSC', 'OWNER', 'P_CRASH2',
        'ROLLOVER','SPEC_USE', 'SPEEDREL', 'STATE', 'TOW_VEH', 'VALIGN', 'VEH_SC1', 'VEH_SC2', 'VINTYPE', 'VPAVETYP', 'VPROFILE',
        'VSURCOND','VTCONT_F', 'VTRAFCON', 'PREV_DWI', 'NUMOCCS', 'PREV_ACC', 'PREV_SPD', 'PREV_SUS', 'VE_FORMS', 'VNUM_LAN',
        'VTRAFWAY', 'FIRE_EXP', 'HAZ_INV', 'HIT_RUN', 'TRAV_SP', 'DEATHS']
full = full[cols]

In [ ]:
# CREATE A NEW BINARY VARIABLE : 0 -if no deaths. 1- if # of deaths >= 1
full['DEATHS_BIN'] = full['DEATHS'].apply(lambda x: 1 if x>=1 else 0)

In [ ]:
full.head()

In [ ]:
print("NUMBER OF DEATHS")
print(full['DEATHS'].unique())
plt.figure(figsize=(9, 8))
plt.hist(full['DEATHS']) #99 - unknown
plt.ylabel('NUMBER OF DEATHS')
plt.show()
print("Mean=",np.mean(full['DEATHS']))
print("Variance=",np.std(full['DEATHS'])**2)

In [ ]:
#full.to_csv('./DataStorage/NHTSA/MISCELLANOUS/full_latest.csv')

In [ ]:
#full = pd.read_csv('./DataStorage/NHTSA/MISCELLANOUS/full_latest.csv',engine='python')

In [ ]:
#full = full.drop(full.columns[full.columns.str.contains('unnamed',case = False)],axis = 1)

# PART 1 : CORRELATION TEST

In [ ]:
def convert(data, to):
    converted = None
    if to == 'array':
        if isinstance(data, np.ndarray):
            converted = data
        elif isinstance(data, pd.Series):
            converted = data.values
        elif isinstance(data, list):
            converted = np.array(data)
        elif isinstance(data, pd.DataFrame):
            converted = data.as_matrix()
    elif to == 'list':
        if isinstance(data, list):
            converted = data
        elif isinstance(data, pd.Series):
            converted = data.values.tolist()
        elif isinstance(data, np.ndarray):
            converted = data.tolist()
    elif to == 'dataframe':
        if isinstance(data, pd.DataFrame):
            converted = data
        elif isinstance(data, np.ndarray):
            converted = pd.DataFrame(data)
    else:
        raise ValueError("Unknown data conversion: {}".format(to))
    if converted is None:
        raise TypeError('cannot handle data conversion of type: {} to {}'.format(type(data),to))
    else:
        return converted

In [ ]:
# CORRELATION BETWEEN ALL THE VARIABLES
import scipy.stats as ss
from collections import Counter
import math

def function(full, nominal_columns=None, **kwargs):
    columns = full.columns
    corr = pd.DataFrame(index=columns, columns=columns) # converts it like a matrix
    if nominal_columns is None:
        nominal_columns = list()
    elif nominal_columns == 'all':
        nominal_columns = columns
    for i in range(0,len(columns)):
            for j in range(i,len(columns)):
                if i == j:
                    corr[columns[i]][columns[j]] = 1.0
                else:
                    if columns[i] in nominal_columns:
                        if columns[j] in nominal_columns:    #BOTH NOMINAL
                            cell = cramers_v(full[columns[i]],full[columns[j]])
                            #result="{} and {} are IMPORTANT = {}".format((i), (j),(cell)) 
                            #print(result)
                            corr[columns[i]][columns[j]] = cell
                            corr[columns[j]][columns[i]] = cell
                        else:                               #1ST - NOMINAL, 2ND - CONTINUOUS
                            cell = correlation_ratio(full[columns[i]], full[columns[j]])
                            corr[columns[i]][columns[j]] = cell
                            corr[columns[j]][columns[i]] = cell
                    else:                                   #1ST - CONTINUOUS, 2ND - NOMINAL
                        if columns[j] in nominal_columns:
                            cell = correlation_ratio(full[columns[j]], full[columns[i]])
                            corr[columns[i]][columns[j]] = cell
                            corr[columns[j]][columns[i]] = cell
                        else:                               #BOTH CONTINUOUS
                            cell, _ = ss.pearsonr(full[columns[i]], full[columns[j]])
                            corr[columns[i]][columns[j]] = cell
                            corr[columns[j]][columns[i]] = cell                    
    #print(corr)
    writer = pd.ExcelWriter('output1.xlsx')
    corr.to_excel(writer,'Sheet1')
    writer.save()
    return corr

def correlation_ratio(categories, measurements): 
    categories = convert(categories, 'array')
    measurements = convert(measurements, 'array')
    fcat, _ = pd.factorize(categories)
    cat_num = np.max(fcat)+1
    y_avg_array = np.zeros(cat_num)
    n_array = np.zeros(cat_num)
    for i in range(0,cat_num):
        cat_measures = measurements[np.argwhere(fcat == i).flatten()]
        n_array[i] = len(cat_measures)
        y_avg_array[i] = np.average(cat_measures)
    y_total_avg = np.sum(np.multiply(y_avg_array,n_array))/np.sum(n_array)
    numerator = np.sum(np.multiply(n_array,np.power(np.subtract(y_avg_array,y_total_avg),2)))
    denominator = np.sum(np.power(np.subtract(measurements,y_total_avg),2))
    if numerator == 0:
        eta = 0.0
    else:
        eta = numerator/denominator
    return eta    
    
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x,y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))
    
nominal_columns = ['ACC_TYPE','BUS_USE','CDL_STAT','DAY','DRIMPAIR','DR_DRINK','DR_HGT','DR_SF1','DR_SF2','DR_SF3','DR_SF4',
          'EMER_USE','HARM_EV','HOUR','HAZ_PLAC','IMPACT1','LAST_YR','L_COMPL','L_RESTRI','L_STATUS','L_TYPE','MAKE','MAN_COLL',
          'MDRDSTRD','MDRMANAV','MFACTOR','MINUTE','MOD_YEAR','MONTH','MVIOLATN','MVISOBSC','OWNER','P_CRASH2','ROLLOVER','SPEC_USE',
          'SPEEDREL','STATE','TOW_VEH','VALIGN','VEH_SC1','VEH_SC2','VINTYPE','VPAVETYP','VPROFILE','VSURCOND','VTCONT_F','VTRAFCON',
           'DR_WGT','GVWR','WEIGHT']
corr = function(full,nominal_columns)

In [ ]:
corr

# OBSERVATIONS:

In [ ]:
GVWR          MAKE      0.429  #REMOVE MAKE
WEIGHT        MAKE      0.431
WEIGHT        VIN_TYPE  0.498
P_CRASH2      ACC_TYPE  0.434
L_TYPE        L_COMPL   0.445  #REMOVE L_TYPE
L_TYPE        L_STATUS  0.464
P_CRASH2      MAN_COLL  0.435  #REMOVE MAN_COLL
STATE         TRAV_SP   0.455
VALIGN        VPAVETYP  0.421  #REMOVE VALIGN
VALIGN        VPROFILE  0.499
VALIGN        VSURCOND  0.420
VEH_SC1       VEH_SC2   0.450  #REMOVE VEH_SC2 

WEIGHT         GVWR     0.563
ACC_TYPE      MAN_COLL  0.576
L_STATUS      L_COMPL   0.558
VIN_TYPE      MAKE      0.593

VTRAFCON      VTCONT_F  0.635
DR_DRINK      DRIMPAIR  0.683  ### DRIMPAIR
HAZ_PLAC      HAZ_INV   1      #REMOVE HAZ_PLAC

In [ ]:
#Removing IRRELEVANT variables from the data
full = full.drop(['MAKE','L_TYPE','MAN_COLL','VALIGN','VEH_SC2','HAZ_PLAC','DRIMPAIR'],axis=1)

In [ ]:
# Create separate dataset for other test
full_data = full.copy() # full_data is actual dataset after correlation

In [ ]:
# Removing COUNT variables from the data
full = full.drop(['VTRAFWAY','NUMOCCS','VNUM_LAN','VE_FORMS','PREV_ACC','PREV_DWI','PREV_SUS','PREV_SPD','DEATHS'], axis = 1)

In [ ]:
full.shape

# PART 2 : CHISQ TEST

In [ ]:
class ChiSquare:
    def __init__(self, dataframe):
        self.df = dataframe
        self.p = None #P-Value
        self.chi2 = None #Chi Test Statistic
        self.dof = None
        
        self.dfObserved = None
        self.dfExpected = None
        
    def TestIndependence(self,colX,colY, alpha=0.05):
        X = self.df[colX].astype(str)
        Y = self.df[colY].astype(str)
        
        self.dfObserved = pd.crosstab(Y,X)  #computes the frequency table of the factors
        chi2, p, dof, expected = stats.chi2_contingency(self.dfObserved.values)  #build contingency table
        self.p = p #'%.2E' % Decimal(p) #np.round(p,5)
        self.chi2 = chi2
        self.dof = dof 
        
        self.dfExpected = pd.DataFrame(expected, columns=self.dfObserved.columns, index = self.dfObserved.index)
        
#Initialize ChiSquare Class
cT = ChiSquare(full_data)
#Feature Selection
testColumns = ['ACC_TYPE', 'BUS_USE', 'CDL_STAT', 'DAY','DR_DRINK', 'DR_HGT', 'DR_SF1', 'DR_SF2', 
               'DR_SF3', 'DR_SF4', 'DR_WGT','EMER_USE', 'FIRE_EXP', 'GVWR', 'HARM_EV', 'HAZ_INV','HIT_RUN', 'HOUR', 
               'IMPACT1', 'LAST_YR', 'L_COMPL', 'L_RESTRI','L_STATUS', 'MDRDSTRD', 'MDRMANAV','MFACTOR', 'MINUTE',
               'MOD_YEAR', 'MONTH', 'MVIOLATN', 'MVISOBSC','OWNER','P_CRASH2', 'ROLLOVER', 'SPEC_USE', 'SPEEDREL', 'STATE', 
               'TOW_VEH','TRAV_SP', 'VEH_SC1', 'VINTYPE','VPAVETYP', 'VPROFILE', 'VSURCOND', 'VTCONT_F', 'VTRAFCON', 'WEIGHT']
for var in testColumns:
    alpha = 0.05
    cT.TestIndependence(colX=var,colY='DEATHS')
    result = ""
    if float(cT.p)<alpha:
        result="{0} is an important predictor.".format(var)
        print(result)
    else:   
        unimportant = []
        x = "{}-{}".format((var),(col))
        print(x)


In [ ]:
# All the variable are statistically significant

In [ ]:
len(testColumns)

In [ ]:
full_data.shape #After removing 6 variables from Correlation algo

In [ ]:
full_1 = full_data.copy()
#Use this dataframe for next part

# PART 3 : FEATURE SELECTION USING LASSO

In [ ]:
full_1.head(5) # DEATHS is the target variable(last one)

In [ ]:
from sklearn.model_selection import train_test_split
def trainTestSplit(data):
    X=np.array(data[list(data.columns[:-2])])
    y = full_1['DEATHS']
    X.shape
    X_train,X_test,y_train,y_test=train_test_split(X,y, random_state=0)
    print(X_train.shape)
    print(X_test.shape)
    print(y_train.shape)
    print(y_test.shape)
    return(X_train,X_test,y_train,y_test)

In [ ]:
X_train,X_test,y_train,y_test = trainTestSplit(full_1)
print("Shape of the dataset ",np.shape(X_train))
print("Size of Data set before feature selection: %.2f MB"%(X_train.nbytes/1e6))

In [ ]:
# Cross Validation for selecting alpha value
feat = full_1.keys().get_values() #Extract attribute names from the data frame
lassocv = LassoCV(alphas=None, cv=10, max_iter=100, normalize=True)
lassocv.fit(X_train, y_train)
lasso = Lasso()
lasso.set_params(alpha=lassocv.alpha_)
print("Alpha=", lassocv.alpha_)
lasso.fit(X_train, y_train)
print("mse = ",mean_squared_error(y_test, lasso.predict(X_test)))
print("best model coefficients:")
lasso_coef = pd.Series(np.round(lasso.coef_,4), index=feat[0:-2])
lasso_coef.to_frame()

In [ ]:
# Features selected from Lasso
full_2 = lasso_coef[(np.absolute(lasso_coef)>0.0000)]
len(full_2) #48
full_2

# PART 4 : FEATURE SELECTION WITH ENET

In [ ]:
#https://www.kaggle.com/cast42/feature-selection-and-elastic-net
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring="mean_squared_error", cv = 10))
    return(rmse)

In [ ]:
l1_ratios = [1.5, 1.1, 1, 0.9, 0.8, 0.7, 0.5]
alphas = [1e-4, 1e-3, 1e-2, 0.05, 1e-1, 1]
cv_elastic = [rmse_cv(ElasticNet(alpha = alpha, l1_ratio=l1_ratio)).mean() for (alpha, l1_ratio) in product(alphas, l1_ratios)]

In [ ]:
#lt.rcParams['figure.figsize'] = (12.0, 6.0)
idx = list(product(alphas, l1_ratios))
p_cv_elastic = pd.Series(cv_elastic, index = idx)
p_cv_elastic.plot(title = "Validation - Just Do It")
plt.xlabel("alpha - l1_ratio")
plt.ylabel("rmse")

In [ ]:
elastic = ElasticNet(alpha=0.0001, l1_ratio=0.7) #(0.0001, 0.5)    0.462328
elastic.fit(X_train, y_train)

In [ ]:
preds = pd.DataFrame({"preds":elastic.predict(X_train), "true":y_train})

In [ ]:
rmse = np.sqrt(np.mean((preds['true']-preds['preds'])**2))
print ('RMSE: {0:.4f}'.format(rmse))
print('R^2 train: %.3f' %  r2_score(preds['true'], preds['preds']))


In [ ]:
coef = pd.Series(np.round(elastic.coef_,4), index = feat[0:-2])
print("Elastic Net picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
coef.to_frame()
# Features selected from Lasso
full_3 = coef[(np.absolute(coef)>0.0000)]
len(full_3) #48
full_3

In [ ]:
#plt.rcParams['figure.figsize'] = (12.0, 8.0)
imp_coef = pd.DataFrame(coef.sort_values())
imp_coef.plot(kind = "barh")
plt.title("Coefficients in the Elastic Net Model")
plt.show()

In [ ]:
full_1 = full_1.drop(['DR_WGT','MINUTE','DAY','DR_HGT','MOD_YEAR','LAST_YR','TRAV_SP'],axis=1)

In [ ]:
full_1.shape

In [ ]:
full_1.columns

In [ ]:
full.shape

# PART 5 : FEATURE SIGNIFICANCE USING XGBOOST MODEL

In [ ]:
X_train,X_test,y_train,y_test = trainTestSplit(full_1) #48

In [ ]:
# INCLUDING ALL THE VARIABLES
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
import xgboost as xg
xg.plot_importance(model)
plt.rcParams['figure.figsize'] = (30.0, 26.0)
plt.yticks(range(len(full_1.columns)), full_1.columns);
plt.show()
# TOP 5 ARE PREV_ACC, NUMOCCS, PREV_DWI, VTRAFCON, VTCONT_F

In [ ]:
full_1 = full_1.drop(['VTRAFWAY','NUMOCCS','VNUM_LAN','VE_FORMS','PREV_ACC','PREV_DWI','PREV_SUS','PREV_SPD'], axis = 1)
full_1.shape

In [ ]:
X_train,X_test,y_train,y_test = trainTestSplit(full_1)

In [ ]:
# INCLUDING NON COUNT VARIABLES
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
xg.plot_importance(model)
plt.rcParams['figure.figsize'] = (30.0, 26.0)
plt.yticks(range(len(full_1.columns)), full_1.columns);
plt.show()

# PART 6 : Poisson Regression model fitting

In [ ]:
#Choosing only the top 5 variables from the non count feature-important set + all count variables + DEATH as target
#https://www.theanalysisfactor.com/count-data-considered-continuous/
var_list = ['VSURCOND','VTRAFCON','VTCONT_F','HIT_RUN','FIRE_EXP','VTRAFWAY','NUMOCCS','VNUM_LAN','VE_FORMS','PREV_ACC','PREV_DWI','PREV_SUS','PREV_SPD','DEATHS']
new_df = full_data.loc[:,var_list]
new_df.shape
#now train - test- validation split

In [ ]:
new_df['DEATHS'].dtype

In [ ]:
new_df

In [ ]:
Xtrain = Xtrain.astype('int')

In [ ]:
ytrain.dtype

In [ ]:
import statsmodels.api as sm
from patsy import dmatrices
formula = """DEATHS ~ VSURCOND+VTRAFCON+VTCONT_F+HIT_RUN+FIRE_EXP+VTRAFWAY+NUMOCCS+VNUM_LAN+VE_FORMS+PREV_ACC+
            PREV_DWI+PREV_SUS+PREV_SPD"""
response, predictors = dmatrices(formula, new_df, return_type='dataframe')


In [ ]:
po_results = sm.GLM(response, predictors, family=sm.families.Poisson()).fit()
print(po_results.summary())

In [ ]:
def ct_response(row):
    "Calculate response observation for Cameron-Trivedi dispersion test"
    y = row['DEATHS']
    m = row['bev_mu']
    return ((y - m)**2 - y) / m

In [ ]:
import statsmodels.formula.api as smf
ct_data = new_df.copy()
ct_data['bev_mu'] = po_results.mu
ct_data['ct_resp'] = ct_data.apply(ct_response, axis=1)
# Linear regression of auxiliary formula
ct_results = smf.ols('ct_resp ~ bev_mu - 1', ct_data).fit()
# Construct confidence interval for alpha, the coefficient of bev_mu
alpha_ci95 = ct_results.conf_int(0.05).loc['bev_mu']
print('\nC-T dispersion test: alpha = {:5.3f}, 95% CI = ({:5.3f}, {:5.3f})'
        .format(ct_results.params[0], alpha_ci95.loc[0], alpha_ci95.loc[1]))

In [ ]:
nb_results = sm.GLM(response, predictors,family=sm.families.NegativeBinomial
                    (alpha=0.671)).fit()
print(nb_results.summary())
#https://dashee87.github.io/football/python/predicting-football-results-with-statistical-modelling/